In [61]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt


In [272]:
df_steam = pd.read_csv('df_EDA_steam_for_ML.csv')
df_steam = df_steam [['id','app_name']]
df_steam

,id,app_name
0,761140.0,Lost Summoner Kitty
1,761140.0,Lost Summoner Kitty
2,761140.0,Lost Summoner Kitty
3,761140.0,Lost Summoner Kitty
4,761140.0,Lost Summoner Kitty
...,...,...
1924517,658870.0,EXIT 2 - Directions
1924518,658870.0,EXIT 2 - Directions
1924519,658870.0,EXIT 2 - Directions
1924520,658870.0,EXIT 2 - Directions


In [273]:
df_reviews = pd.read_csv('df_EDA_reviews_for_ML.csv')
df_reviews = df_reviews[['user_id', 'id', 'sentiment']]
df_reviews

,user_id,id,sentiment
0,76561197970982479,1250.0,2
1,76561197970982479,22200.0,2
2,76561197970982479,43110.0,2
3,js41637,251610.0,2
4,js41637,227300.0,0
...,...,...,...
48466,wayfeng,730.0,1
48467,76561198251004808,253980.0,2
48468,72947282842,730.0,0
48469,ApxLGhost,730.0,2


In [274]:
df_reviews = df_reviews[['user_id', 'id', 'sentiment']].merge(df_steam[['id', 'app_name']], on='id', how='left')

In [275]:
df_reviews

,user_id,id,sentiment,app_name
0,76561197970982479,1250.0,2,Killing Floor
1,76561197970982479,1250.0,2,Killing Floor
2,76561197970982479,1250.0,2,Killing Floor
3,76561197970982479,1250.0,2,Killing Floor
4,76561197970982479,1250.0,2,Killing Floor
...,...,...,...,...
13495767,ApxLGhost,730.0,2,Counter-Strike: Global Offensive
13495768,ApxLGhost,730.0,2,Counter-Strike: Global Offensive
13495769,ApxLGhost,730.0,2,Counter-Strike: Global Offensive
13495770,ApxLGhost,730.0,2,Counter-Strike: Global Offensive


In [276]:
df_reviews = df_reviews.drop_duplicates()

In [277]:
df_reviews = df_reviews.rename(columns={'id': 'item_id'})


In [278]:
df_reviews = df_reviews.dropna(subset='app_name')

In [279]:
df_reviews['app_name'] = df_reviews['app_name'].astype(str)

In [280]:
df_reviews 

,user_id,item_id,sentiment,app_name
0,76561197970982479,1250.0,2,Killing Floor
200,76561197970982479,22200.0,2,Zeno Clash
306,js41637,227300.0,0,Euro Truck Simulator 2
546,js41637,239030.0,0,"Papers, Please"
786,evcentric,370360.0,2,TIS-100
...,...,...,...,...
13495080,76561198239215706,730.0,1,Counter-Strike: Global Offensive
13495240,wayfeng,730.0,1,Counter-Strike: Global Offensive
13495400,76561198251004808,253980.0,2,Enclave
13495451,72947282842,730.0,0,Counter-Strike: Global Offensive


In [282]:
df = df_reviews
df.to_csv('df_ML.csv')


Filtrado Colaborativo para el sistema de recomendación user-item
Uso la biblioteca Surprise para entrenar el modelo.


    Use a custom dataset
    Surprise has a set of builtin datasets, but you can of course use a custom dataset. Loading a rating dataset can be done either from a file (e.g. a csv file), or from a pandas dataframe. To load a dataset from a pandas dataframe, you will need the load_from_df() method. You will also need a Reader object, but only the rating_scale parameter must be specified. The dataframe must have three columns, corresponding to the user (raw) ids, the item (raw) ids, and the ratings in this order. Each row thus corresponds to a given rating. This is not restrictive as you can reorder the columns of your dataframe easily.

    

In [283]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy, SVD
from surprise.model_selection import train_test_split
import joblib
from surprise import SVD
from surprise.model_selection import cross_validate

In [284]:
df = pd.read_csv('df_ML.csv')
df

,Unnamed: 0,user_id,item_id,sentiment,app_name
0,0,76561197970982479,1250.0,2,Killing Floor
1,200,76561197970982479,22200.0,2,Zeno Clash
2,306,js41637,227300.0,0,Euro Truck Simulator 2
3,546,js41637,239030.0,0,"Papers, Please"
4,786,evcentric,370360.0,2,TIS-100
...,...,...,...,...,...
40224,13495080,76561198239215706,730.0,1,Counter-Strike: Global Offensive
40225,13495240,wayfeng,730.0,1,Counter-Strike: Global Offensive
40226,13495400,76561198251004808,253980.0,2,Enclave
40227,13495451,72947282842,730.0,0,Counter-Strike: Global Offensive


Instanciamos la clase Reader() y definimos la escala de sentiment

In [285]:
reader = Reader(rating_scale=(df['sentiment'].min(), df['sentiment'].max()))

Abro el Dataset usando Dataset.load_from_df

In [286]:
data = Dataset.load_from_df(df[['user_id', 'item_id', 'sentiment']], reader)

#data = Dataset.load_from_file(file_path, reader=reader) otra opcion


Train-test split and the fit() method
If you don’t want to run a full cross-validation procedure, you can use the train_test_split() to sample a trainset and a testset with given sizes, and use the accuracy metric of your chosing. You’ll need to use the fit() method which will train the algorithm on the trainset, and the test() method which will return the predictions made from the testset:

Separo el data con tran_test_split, dejando un 30% de la data para test.

In [307]:
trainset, testset = train_test_split(data, test_size=.30)

Entrenamiento Opcion I

In [308]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)



RMSE: 0.8060


0.8059812215572201

Entrenamiento Opcion II

evaluación del rendimiento de un algoritmo  SVD (Singular Value Decomposition) en un proceso de validación cruzada con 3 divisiones (folds). La validación cruzada se utiliza comúnmente en machine learning para evaluar el rendimiento de un modelo de manera más robusta

In [289]:
#creo svd
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8098  0.7984  0.8042  0.8041  0.0047  
MAE (testset)     0.7184  0.7098  0.7130  0.7138  0.0036  
Fit time          0.25    0.27    0.28    0.27    0.01    
Test time         0.12    0.12    0.10    0.12    0.01    


{'test_rmse': array([0.80982732, 0.79839574, 0.80421531]),
 'test_mae': array([0.7184348 , 0.70981181, 0.71303112]),
 'fit_time': (0.25205016136169434, 0.2655019760131836, 0.2809407711029053),
 'test_time': (0.12261962890625, 0.11882567405700684, 0.10443878173828125)}

Entrenamiento del modelo SVD


Generación de predicciones de calificación/score.
    
    Predecimos sobre el testset, ya que no lo conoce el modelo creado SVD

In [290]:
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [291]:
predictions = svd.test(testset)


In [292]:
print(predictions [1])
print(predictions [100])

user: 76561198085788881 item: 322170.0   r_ui = 2.00   est = 1.40   {'was_impossible': False}
user: 76561198165512984 item: 730.0      r_ui = 0.00   est = 1.36   {'was_impossible': False}


Guardamos el modelo desarrollado


In [309]:


joblib.dump(algo, 'modelo_svd_I.pkl')


# Suponiendo que 'modelo_svd' es tu modelo entrenado SVD
joblib.dump(svd, 'modelo_svd_II.pkl')



['modelo_svd_II.pkl']

Usar el modelo  para predecir la calificación de una juego

In [310]:
def  recomendacion_usuario(user_id:str):
    modelo_svd = joblib.load('modelo_svd_I.pkl')
    df
    top=5

    """
   Esta funcion devuelve los 5 videojuegos que le gustaran al usuario ingresado 
   del DataFrame user_reviews de Steam Games.

    Args:
    - user_id: id del usuario para el cual se generarán las recomendaciones

    Returns:
    - lista de 5 videojuegos recomendados para el usuario elegido
    """

    # Obtener las películas que el usuario no ha visto aún
    games_play = set(df[df['user_id'] == user_id]['item_id'])
    games_all = set(df['item_id'])
    games_unplay = list(games_all - games_play)

    # Obtener las recomendaciones
    predicted_ratings = [modelo_svd.predict(user_id, item_id).est for item_id in games_unplay]

    # Ordenar los juegos según su predicción de rating
    games_rating = list(zip(games_unplay, predicted_ratings))
    games_rating.sort(key=lambda x: x[1], reverse=True)

    # Obtener los títulos de los juegos recomendadas
    recommended_games = games_rating[:top]
    recommended_games = [df[df['item_id'] == item_id]['app_name'].iloc[0] for item_id, _ in recommended_games]

    return {
        "recomended_games": recommended_games,
        "user_id": user_id,
        "top": 5
    }

In [235]:
#recomendacion_usuario(76561198069536565)
#opcion dos deberia cambiar el nombre del archivo pkl en la funcion  
#'modelo_svd_II.pkl'

{'recomended_games': ['Transistor',
  'Bastion',
  'The Wolf Among Us',
  'Shadow Warrior',
  'RIFT'],
 'user_id': 76561198069536565,
 'top': 5}

In [311]:
recomendacion_usuario( 76561198069536565)

{'recomended_games': ['Gunpoint',
  "King Arthur's Gold",
  'RIFT',
  "The Witcher: Enhanced Edition Director's Cut",
  'Trine 2: Complete Story'],
 'user_id': 76561198069536565,
 'top': 5}